# Model Definition and Evaluation
## Table of Contents
1. [Model Selection](#model-selection)
2. [Feature Engineering](#feature-engineering)
3. [Hyperparameter Tuning](#hyperparameter-tuning)
4. [Implementation](#implementation)
5. [Evaluation Metrics](#evaluation-metrics)
6. [Comparative Analysis](#comparative-analysis)


## Setup
### Imports

In [19]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import matplotlib.pyplot as pl
import os, json, hashlib

from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from huggingface_hub import login, list_repo_files, hf_hub_download, upload_file

### Configuration

In [ ]:
REPO_ID = "mttfst/Paulette_Cloud_Tracks"
token = ""

WINDOW_MINUTES =  30 #Minuten
CUTOFF_STEPS = 5

MODEL = "SimpleRNN"
LOSS = "MSE"

BATCH_SIZE = 8
EPOCHS = 2 #40

In [ ]:
# =========================================
# Login HuggingFace
# =========================================
login(token)

### Logging

In [ ]:
# Du kannst hier ein separates Repo für Logs/Configs nutzen (empfohlen),
# oder du lässt es auf dem Dataset-Repo laufen.
CONFIG_REPO_ID = REPO_ID  # z.B. "thorsten789/hurricane_cloud_runs"

def make_run_id(prefix: str, config: dict) -> str:
    ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    cfg_str = json.dumps(config, sort_keys=True)
    h = hashlib.sha1(cfg_str.encode("utf-8")).hexdigest()[:10]
    return f"{prefix}_{ts}_{h}"

def save_json_local(path: str, data: dict) -> str:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    return path

def upload_json_hf(local_path: str, run_id: str, name: str, base_dir: str = "runs"):
    """Lädt JSON als {base_dir}/{run_id}/{name}.json in CONFIG_REPO_ID hoch."""
    try:
        path_in_repo = f"{base_dir}/{run_id}/{name}.json"
        upload_file(
            path_or_fileobj=local_path,
            path_in_repo=path_in_repo,
            repo_id=CONFIG_REPO_ID,
            repo_type="dataset",
            commit_message=f"Add {name}.json for {run_id}",
        )
        print(f"[HF] uploaded: {path_in_repo}")
    except Exception as e:
        print(f"[HF] upload skipped/failed ({name}): {e}")

# --- zentrale Run-Config (die ID basiert auf config -> sinnvoller Run-Name)
RUN_CONFIG = {
    "model": MODEL,
    "optimizer": "adam",
    "units1": 64,
    "units2": 32,
    "lr": 1e-3,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS,
    "cutoff_steps": CUTOFF_STEPS,
    "loss": LOSS,
}

RUN_ID = make_run_id("simple_rnn", RUN_CONFIG)
print("RUN_ID:", RUN_ID)

# Setup sofort speichern (damit du schon am Anfang einen Run hast)
setup = {
    "run_id": RUN_ID,
    "config": RUN_CONFIG,
    "data": {"repo_id": REPO_ID},
    "meta": {"notebook": "3_Model/model_definition_evaluation_JS.ipynb"},
}

# +++ Logging: Save Setup +++
local_setup = save_json_local(f"runs_local/{RUN_ID}/setup.json", setup)
upload_json_hf(local_setup, RUN_ID, "setup")

class AutoSaveTrain(tf.keras.callbacks.Callback):
    """Speichert train.json am Ende von model.fit (History + best_val_loss)."""
    def __init__(self, run_id: str):
        super().__init__()
        self.run_id = run_id

    def on_train_end(self, logs=None):
        hist = getattr(self.model, "history", None)
        history_dict = hist.history if hist is not None else {}

        train_data = {
            "run_id": self.run_id,
            "history": history_dict,
            "summary": {
                "best_val_loss": float(min(history_dict["val_loss"])) if "val_loss" in history_dict else None,
                "final_train_loss": float(history_dict["loss"][-1]) if "loss" in history_dict and len(history_dict["loss"]) else None,
            },
        }
        local_train = save_json_local(f"runs_local/{self.run_id}/train.json", train_data)
        upload_json_hf(local_train, self.run_id, "train")


RUN_ID: simple_rnn_2026-02-15_00-39-37_a7afa260af
[HF] uploaded: runs/simple_rnn_2026-02-15_00-39-37_a7afa260af/setup.json


### Load Data from HuggingFace

TRACK LENGTH FILE:

Diese CSV enthält Metadaten über alle Wolken:
- filename: Dateiname des Tracks
- track_len: Anzahl Zeitschritte (Lebensdauer)

WARUM WICHTIG?
- Wir trainieren nur auf Wolken mit >= 120 Zeitschritten
- Grund: Kurze Wolken (<60 Minuten) sind zu variabel/chaotisch
- Längere Wolken zeigen klare Lebenszyklen

DATENQUELLE:
Diese Datei wurde vorberechnet aus allen Tracks.
Spart Zeit beim Training-Setup.

In [22]:
tracks_120 = None

# Wenn lokal nicht da: von Hugging Face Dataset-Repo herunterladen
if tracks_120 is None:
    from huggingface_hub import hf_hub_download

    filename_in_repo = "track_len/track_len_exp_1.1.csv"

    print(f"⬇️  Downloading from HF: {REPO_ID}/{filename_in_repo}")
    local_file = hf_hub_download(
        repo_id=REPO_ID,
        repo_type="dataset",
        filename=filename_in_repo,
    )

    print(f"✅ Loading track_len from HF-cached file: {local_file}")
    tracks_120 = pd.read_csv(local_file)

print("tracks_120 shape:", tracks_120.shape)
tracks_120.head()

# =========================================
# Dataset-Split
# =========================================
files = list_repo_files(REPO_ID, repo_type="dataset")
csv_files = [f for f in files if f.startswith("exp_1.1/") and f.endswith(".csv")]

tracks_120 = tracks_120[tracks_120.track_len >= 120]
print("Total CSV tracks with at least 120 timesteps:", len(tracks_120))

tracks_120 = tracks_120.filename.to_list()

csv_files = [
    f for f in csv_files
    if f.split("/")[1] in tracks_120
]

random.seed(42)
random.shuffle(csv_files)

n = len(csv_files)
#train_files = csv_files[: int(0.7 * n)]
#val_files   = csv_files[int(0.7 * n): int(0.85 * n)]
#test_files  = csv_files[int(0.85 * n):]

train_files= csv_files[:150]
val_files= csv_files[100:125]
test_files= csv_files[150:175]

print(f"Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")

⬇️  Downloading from HF: mttfst/Paulette_Cloud_Tracks/track_len/track_len_exp_1.1.csv
✅ Loading track_len from HF-cached file: /root/.cache/huggingface/hub/datasets--mttfst--Paulette_Cloud_Tracks/snapshots/8248ff1af359eabb4055f7d98300dd3810e359c3/track_len/track_len_exp_1.1.csv
tracks_120 shape: (9227, 2)
Total CSV tracks with at least 120 timesteps: 1115
Train: 150, Val: 25, Test: 25


## Model Selection

[Discuss the type(s) of models you consider for this task, and justify the selection.]



## Feature Engineering

[Describe any additional feature engineering you've performed beyond what was done for the baseline model.]


In [23]:
PROFILE_PREFIXES = ["qr_", "qc_", "qi_", "qs_", "qg_", "qv_", "roh_", "w_"]

SCALAR_FEATURES = [
    "cape_ml_L00", "cin_ml_L00",
    "lwp_L00",
    "iwp_L00", "rain_gsp_rate_L00",
    "tqc_L00", "tqi_L00", "area_m2"
]

In [24]:
def compute_remaining_lifetime(df, timestep_minutes=5):
    """
    Berechnet verbleibende Lebensdauer pro Zeitschritt
    """
    n = len(df)
    return [(n - i - 1) * timestep_minutes for i in range(n)]

#def compute_future_rain(df, timestep_minutes=5):
#    rain = df["rain_gsp_rate_L00"].values
#    dt = timestep_minutes * 60  # Sekunden

#    future_rain = []
#    for i in range(len(rain)):
#        future_rain.append(rain[i:].sum() * dt)

#    return future_rain

In [25]:
def extract_profile(df, prefix, n_levels=50):
    """
    Extrahiert ein vertikales Profil mit exakt n_levels.
    Fehlende Level werden mit 0 aufgefüllt.
    """
    data = np.zeros((len(df), n_levels), dtype="float32")
    for i in range(n_levels):
        col = f"{prefix}L{i:02d}"
        if col in df.columns:
            data[:, i] = df[col].values
    return data

In [26]:
def extract_ts_features_from_profiles(profiles):
    """
    profiles: (T, Z, F) mit F = len(PROFILE_PREFIXES)
    Level 50=unten → Level 0=oben
    """
    idx = {name: i for i, name in enumerate(PROFILE_PREFIXES)}
    
    T, Z = profiles.shape[:2]
    levels = np.arange(Z-1, -1, -1, dtype="float32")  # 49...0 (unten→oben)
    
    ts_features = []
    
    for t in range(T):
        # Profile extrahieren
        qc_t = profiles[t, :, idx["qc_"]]  # (50,)
        qi_t = profiles[t, :, idx["qi_"]]
        qr_t = profiles[t, :, idx["qr_"]]
        w_t  = profiles[t, :, idx["w_"]]
        
        # Wolkenmaske (qc+qi > Schwellwert)
        cloud_mask = (qc_t + qi_t) > 1e-10
        
        if not np.any(cloud_mask):
            ts_features.append(np.zeros(12, dtype="float32"))
            continue
        
        # 1. GEOMETRIE
        cloud_levels = levels[cloud_mask]
        cloud_base   = cloud_levels.max()   # höchster Level = unten
        cloud_top    = cloud_levels.min()   # niedrigster Level = oben
        cloud_thickness = cloud_base - cloud_top
        
        # 2. MASSE
        cloud_mass = np.sum(qc_t + qi_t)
        rain_mass  = np.sum(qr_t)
        
        # 3. DYNAMIK (nur in Wolke)
        w_in_cloud = w_t[cloud_mask]
        mean_w = np.mean(w_in_cloud)
        max_w  = np.max(w_in_cloud)
        
        # 4. HÖHENINFO (Level-Indizes 0-49)
        height_max_qc = np.argmax(qc_t)
        height_max_w  = np.argmax(w_t)
        
        # 5. EXTRA (wie in deinem Code)
        center_of_mass = np.sum(levels * (qc_t + qi_t)) / max(cloud_mass, 1e-12)
        max_qc = np.max(qc_t)
        std_w_cloud = np.std(w_in_cloud)
        
        ts_features.append([
            cloud_base, cloud_top, cloud_thickness,      # Geometrie (3)
            cloud_mass, rain_mass,                       # Masse (2)  
            mean_w, max_w,                               # Dynamik (2)
            height_max_qc, height_max_w,                 # Höheninfo (2)
            center_of_mass, max_qc, std_w_cloud          # Extra (3)
        ])
    
    return np.array(ts_features, dtype="float32")


In [27]:
def preprocess_cloud(df):
    df = df.sort_values("time")

    # Target
    y_lifetime = compute_remaining_lifetime(df)

    # Profile (T, Z, F)
    profile_features = []
    for prefix in PROFILE_PREFIXES:
        prof = extract_profile(df, prefix, n_levels=50)
        profile_features.append(prof)
    
    profiles = np.stack(profile_features, axis=-1)
    
    # Scalars
    scalars = df[SCALAR_FEATURES].values.astype("float32")
    
    # neue TS-Features
    ts_features = extract_ts_features_from_profiles(profiles)

    return {
        "ts_features": ts_features,      # (T, 12)
        "scalars": scalars,              # (T, 8)
        "y": np.array(y_lifetime, dtype="float32")[:, None]
    }

In [28]:
def load_and_preprocess(files):
    samples = []
    for f in files:
        local_file = hf_hub_download(
            repo_id=REPO_ID,
            repo_type="dataset",
            filename=f,
        )
        df = pd.read_csv(local_file)
        
        if len(df) <= CUTOFF_STEPS:
            continue
        
        sample = preprocess_cloud(df)
        samples.append(sample)
    
    return samples

In [29]:
print("Loading data...")
train_samples = load_and_preprocess(train_files)
val_samples   = load_and_preprocess(val_files)
test_samples  = load_and_preprocess(test_files)

print(f"Loaded: {len(train_samples)} train, {len(val_samples)} val, {len(test_samples)} test")

Loading data...
Loaded: 150 train, 25 val, 25 test


In [30]:
def create_fixed_window_sequences(samples, window_steps=WINDOW_MINUTES):  # 30min
    X, y = [], []
    WINDOW_STEPS = int(WINDOW_MINUTES * 60 / 30)  # 30min * 60s/min / 30s/step = 60

    for s in samples:
        combined = np.concatenate([s["scalars"], s["ts_features"]], axis=1)
        for i in range(len(combined) - WINDOW_STEPS):
            X.append(combined[i:i+WINDOW_STEPS])  # (60, 20)
            y.append(s["y"][i+WINDOW_STEPS])  # RUL am Ende
    return np.array(X), np.array(y)

X_train, y_train = create_fixed_window_sequences(train_samples, WINDOW_MINUTES)
X_val, y_val = create_fixed_window_sequences(val_samples, WINDOW_MINUTES)
X_test, y_test = create_fixed_window_sequences(test_samples, WINDOW_MINUTES)

In [31]:
sample = train_samples[0]
ts_features = sample["ts_features"]

print("cloud_base  > cloud_top?", np.all(ts_features[:,0] >= ts_features[:,1]))
print("thickness > 0?", np.all(ts_features[:,2] > 0))
print("Sample values:\n", ts_features[:3, :5])  # Erste 5 Features

cloud_base  > cloud_top? True
thickness > 0? True
Sample values:
 [[4.2000000e+01 5.0000000e+00 3.7000000e+01 8.6598527e-03 1.0412185e-02]
 [4.2000000e+01 5.0000000e+00 3.7000000e+01 9.4145974e-03 1.1317367e-02]
 [4.2000000e+01 5.0000000e+00 3.7000000e+01 1.0146300e-02 1.2264996e-02]]


In [32]:
sample = train_samples[8]

scalars = sample["scalars"]          # (T, 8)
ts      = sample["ts_features"]      # (T, 12)

# Zu (T, 20) zusammenfügen
combined = np.concatenate([scalars, ts], axis=1)

cols = [
    "cape_ml_L00", "cin_ml_L00",
    "lwp_L00",
    "iwp_L00", "rain_gsp_rate_L00",
    "tqc_L00", "tqi_L00", "area_m2",
    "cloud_base", "cloud_top", "cloud_thickness",
    "cloud_mass", "rain_mass",
    "mean_w", "max_w",
    "height_max_qc", "height_max_w",
    "center_of_mass", "max_qc", "std_w_in_cloud"
]

ts_df = pd.DataFrame(combined, columns=cols)
ts_df.head()

,cape_ml_L00,cin_ml_L00,lwp_L00,iwp_L00,rain_gsp_rate_L00,tqc_L00,tqi_L00,area_m2,cloud_base,cloud_top,cloud_thickness,cloud_mass,rain_mass,mean_w,max_w,height_max_qc,height_max_w,center_of_mass,max_qc,std_w_in_cloud
0,NaN,NaN,2.507058,0.188360,0.000228,2.034199,0.037669,31360000.0,42.0,8.0,34.0,0.008732,0.002572,0.271322,1.020461,30.0,36.0,16.148663,0.001669,0.420280
1,NaN,NaN,2.567386,0.176616,0.000240,2.066459,0.037015,31360000.0,42.0,7.0,35.0,0.008874,0.002705,0.281104,1.030474,30.0,36.0,16.120064,0.001720,0.419132
2,NaN,NaN,2.617499,0.166323,0.000243,2.089409,0.036390,31360000.0,42.0,7.0,35.0,0.008977,0.002832,0.284971,1.034724,30.0,36.0,16.089861,0.001759,0.421756
3,NaN,NaN,2.656605,0.157891,0.000245,2.102927,0.035792,31360000.0,42.0,7.0,35.0,0.009041,0.002950,0.286870,1.033336,30.0,36.0,16.061790,0.001787,0.421885
4,NaN,NaN,2.683506,0.151085,0.000247,2.106670,0.035219,70560000.0,42.0,7.0,35.0,0.009063,0.003056,0.286429,1.024991,30.0,36.0,16.037300,0.001802,0.418588


### Normalisierung

In [33]:
print("Start Normalization")

scaler = StandardScaler()
X_train_flat = X_train.reshape(-1, X_train.shape[-1])  # (4500, 12)
X_train_scaled = scaler.fit_transform(X_train_flat).reshape(X_train.shape)

X_val_flat = X_val.reshape(-1, X_val.shape[-1])
X_val_scaled = scaler.transform(X_val_flat).reshape(X_val.shape)

X_test_flat = X_test.reshape(-1, X_test.shape[-1])
X_test_scaled = scaler.transform(X_test_flat).reshape(X_test.shape)

print("TS alle Splits skaliert")

print(f"Final shapes:")
print(f"  Train: TS={X_train_scaled.shape}")
print(f"  Val:   TS={X_val_scaled.shape}")
print(f"  Test:  TS={X_test_scaled.shape}")


Start Normalization
TS alle Splits skaliert
Final shapes:
  Train: TS=(23528, 60, 20)
  Val:   TS=(4580, 60, 20)
  Test:  TS=(3646, 60, 20)


## Hyperparameter Tuning

[Discuss any hyperparameter tuning methods you've applied, such as Grid Search or Random Search, and the rationale behind them.]


In [34]:
# Implement hyperparameter tuning
# Example using GridSearchCV with a DecisionTreeClassifier
# param_grid = {'max_depth': [2, 4, 6, 8]}
# grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
# grid_search.fit(X_train, y_train)


## Implementation

[Implement the final model(s) you've selected based on the above steps.]


## Evaluation Metrics

[Clearly specify which metrics you'll use to evaluate the model performance, and why you've chosen these metrics.]


## Comparative Analysis

[Compare the performance of your model(s) against the baseline model. Discuss any improvements or setbacks and the reasons behind them.]


In [35]:
# Comparative Analysis code (if applicable)
# Example: comparing accuracy of the baseline model and the new model
# print(f"Baseline Model Accuracy: {baseline_accuracy}, New Model Accuracy: {new_model_accuracy}")
